In [1]:
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from ufl import ds, dx, grad, inner

from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import pyvista

In [38]:
L = 1
W = 0.2
mu = 1
rho = 1
delta = W/L
gamma = 0.4*delta**2
beta = 1.25
lambda_ = beta
g = gamma

In [40]:
# Create mesh and define function space
# mesh = BoxMesh(Point(0, 0, 0), Point(L, W, W), 10, 3, 3)
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (L, W)), n=(32, 16),
                            cell_type=mesh.CellType.triangle,)

V = fem.VectorFunctionSpace(msh, ("Lagrange", 1))

In [41]:
facets = mesh.locate_entities_boundary(msh, dim=1,
                                       marker=lambda x: np.isclose(x[0], 0.0))

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=fem.Constant(msh, (ScalarType(0), ScalarType(0))), dofs=dofs, V=V)

In [42]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

def epsilon(u):
    # return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
    return ufl.sym(ufl.nabla_grad(u))

def sigma(u):
    return lambda_*ufl.nabla_div(u)*ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

# Define variational problem
f = fem.Constant(msh, (ScalarType(0), ScalarType(-rho*g)))
T = fem.Constant(msh, (ScalarType(0), ScalarType(0)))
a = inner(sigma(u), epsilon(v))*dx
L = inner(f, v)*dx + inner(T, v)*ds

problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [43]:
with io.XDMFFile(MPI.COMM_SELF, "mesh_rank_{}.xdmf".format(MPI.COMM_WORLD.rank), "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)

In [ ]:
cells, types, x = plot.create_vtk_mesh(msh, 2)
grid = pyvista.UnstructuredGrid(cells, types, x)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
# plotter.show()
# if not pyvista.OFF_SCREEN:
#     plotter.show()
# else:
plotter.screenshot("tmp.png")
# pyvista.start_xvfb()
# figure = plotter.screenshot("fundamentals_mesh.png")